In [24]:
!pip install lazypredict scikit-learn --quiet

In [25]:
import pandas as pd
from lazypredict.Supervised import LazyClassifier, LazyRegressor
from sklearn.model_selection import train_test_split

In [26]:
df = pd.read_csv(r"uscecchini28.csv")
df.tail()

,fyear,gvkey,sich,insbnk,understatement,option,p_aaer,new_p_aaer,misstate,act,...,soft_assets,ch_cs,ch_cm,ch_roa,issue,bm,dpi,reoa,EBIT,ch_fcf
146040,2014,314866,8200.00,0,0,0,NaN,NaN,0,262.60,...,0.75,0.56,0.13,-0.05,1,0.10,0.83,-0.33,-0.01,-0.26
146041,2014,315318,2890.00,0,0,0,NaN,NaN,0,1578.40,...,0.74,-0.12,0.03,0.10,1,0.58,0.74,-0.08,0.00,-0.30
146042,2014,316056,3420.00,0,0,0,NaN,NaN,0,973.80,...,0.75,0.00,-0.04,0.07,1,-0.00,1.06,-0.00,0.15,0.07
146043,2014,317260,4412.00,0,0,0,NaN,NaN,0,51.74,...,0.02,NaN,NaN,NaN,1,1.11,NaN,0.00,0.03,NaN
146044,2014,317264,4412.00,0,0,0,NaN,NaN,0,233.21,...,0.07,1.68,-0.09,0.02,1,1.15,0.75,0.03,0.03,-0.24


In [50]:
y = df['misstate']

# Drop Fraud-Specific Labels
X = df.drop(['misstate','p_aaer','new_p_aaer','understatement'], axis=1)
X = X[['at', 'ap', 'ceq', 'che', 'csho', 'dltt', 'dp', 'ni', 'ppegt', 'pstk',
      're', 'rect', 'sale', 'lt', 'xint', 'ivao', 'ch_rsst', 'dch_rec', 'ch_cs', 'ch_cm', 'ch_roa', 'dpi', 'reoa'
     ]]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)

In [55]:
X.head()


,at,ap,ceq,che,csho,dltt,dp,ni,ppegt,pstk,...,lt,xint,ivao,ch_rsst,dch_rec,ch_cs,ch_cm,ch_roa,dpi,reoa
0,32.34,3.74,6.26,0.00,2.53,17.69,1.97,2.13,31.77,0.00,...,26.07,2.33,0.00,0.05,0.04,0.10,0.08,-0.02,0.87,0.17
1,7.78,0.80,0.67,0.17,3.56,5.87,0.70,-1.87,7.33,0.00,...,7.12,0.64,0.13,-0.24,-0.01,0.19,-0.21,-0.12,0.75,-0.43
2,118.12,3.60,44.39,3.13,3.88,37.60,6.27,2.57,78.33,0.00,...,73.73,3.96,12.99,0.02,0.00,0.10,-0.11,0.09,1.02,0.39
3,34.59,3.95,7.75,0.41,4.75,8.91,1.32,0.42,11.14,1.29,...,25.55,2.27,0.00,0.04,0.04,-0.01,-0.25,0.02,1.03,0.09
4,27.54,3.52,-12.14,1.02,6.74,0.27,2.71,-22.81,5.78,0.00,...,39.68,3.51,0.00,-0.58,-0.10,-0.23,-1.67,-0.47,0.60,-0.94


In [36]:
from lazypredict.Supervised import CLASSIFIERS
print(CLASSIFIERS)

[('AdaBoostClassifier', <class 'sklearn.ensemble._weight_boosting.AdaBoostClassifier'>), ('BaggingClassifier', <class 'sklearn.ensemble._bagging.BaggingClassifier'>), ('BernoulliNB', <class 'sklearn.naive_bayes.BernoulliNB'>), ('CalibratedClassifierCV', <class 'sklearn.calibration.CalibratedClassifierCV'>), ('CategoricalNB', <class 'sklearn.naive_bayes.CategoricalNB'>), ('DecisionTreeClassifier', <class 'sklearn.tree._classes.DecisionTreeClassifier'>), ('DummyClassifier', <class 'sklearn.dummy.DummyClassifier'>), ('ExtraTreeClassifier', <class 'sklearn.tree._classes.ExtraTreeClassifier'>), ('ExtraTreesClassifier', <class 'sklearn.ensemble._forest.ExtraTreesClassifier'>), ('FixedThresholdClassifier', <class 'sklearn.model_selection._classification_threshold.FixedThresholdClassifier'>), ('GaussianNB', <class 'sklearn.naive_bayes.GaussianNB'>), ('KNeighborsClassifier', <class 'sklearn.neighbors._classification.KNeighborsClassifier'>), ('LabelPropagation', <class 'sklearn.semi_supervised._

In [52]:
from lazypredict.Supervised import LazyClassifier
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.ensemble._weight_boosting import AdaBoostClassifier
from sklearn.ensemble._bagging import BaggingClassifier
from sklearn.naive_bayes import GaussianNB, CategoricalNB, BernoulliNB

# Pass the classifier classes directly in a list.
selected_classifiers = [XGBClassifier, DecisionTreeClassifier, ExtraTreeClassifier,
                        AdaBoostClassifier, BaggingClassifier,
                        GaussianNB, CategoricalNB, BernoulliNB, ]

clf = LazyClassifier(predictions=True, classifiers=selected_classifiers)
models, predictions = clf.fit(X_train, X_test, y_train, y_test)

  0%|          | 0/8 [00:00<?, ?it/s]

In [53]:
models

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
BernoulliNB,0.89,0.54,0.54,0.93,0.17
DecisionTreeClassifier,0.99,0.54,0.54,0.99,7.31
GaussianNB,0.96,0.53,0.53,0.98,0.14
ExtraTreeClassifier,0.99,0.52,0.52,0.99,0.20
BaggingClassifier,0.99,0.51,0.51,0.99,48.54
XGBClassifier,0.99,0.50,0.50,0.99,1.67
AdaBoostClassifier,0.99,0.50,0.50,0.99,24.35


In [84]:
import yfinance as yf
import pandas as pd

def get_fraud_variables_and_ratios(ticker_symbol):
    """
    Downloads financial data from yfinance and calculates a subset of variables
    and ratios from the uscecchini28 accounting fraud dataset.

    Args:
        ticker_symbol (str): The ticker symbol for the company on Yahoo Finance.

    Returns:
        dict: A dictionary containing the extracted raw variables and calculated
              ratios. Returns None if data retrieval fails or is insufficient.
    """
    try:
        # 1. Download the data from yfinance
        stock = yf.Ticker(ticker_symbol)
        balance_sheet = stock.balance_sheet
        income_statement = stock.financials

        # Transpose the DataFrames for easier year-over-year calculations
        bs_T = balance_sheet.T
        is_T = income_statement.T

    except Exception as e:
        print(f"Error: Could not retrieve data for {ticker_symbol}. Details: {e}")
        return None

    # Check for sufficient data
    if len(bs_T.columns) < 2 or len(is_T.columns) < 2:
        print("Error: Insufficient data to perform year-over-year calculations.")
        return None

    # Extract the most recent two years of data
    bs_2024 = bs_T.iloc[0]
    bs_2023 = bs_T.iloc[1]
    is_2024 = is_T.iloc[0]
    is_2023 = is_T.iloc[1]

    # Dictionary to store the extracted variables and calculated ratios
    results = {}

    # --- 2. Extracting Raw Variables ---
    try:
        # Mappings from uscecchini28 dataset to yfinance names
        #results['fyear'] = bs_2024.name.year
        results['at'] = bs_2024.get('Total Assets')
        results['ap'] = bs_2024.get('Accounts Payable')
        results['ceq'] = bs_2024.get('Common Stock Equity')
        results['che'] = bs_2024.get('Cash And Cash Equivalents')
        results['csho'] = bs_2024.get('Ordinary Shares Number')
        results['dltt'] = bs_2024.get('Long Term Debt And Capital Lease Obligation')
        results['dp'] = is_2024.get('Reconciled Depreciation')
        results['ni'] = is_2024.get('Net Income Common Stockholders')
        results['ppegt'] = bs_2024.get('Gross PPE')
        results['pstk'] = bs_2024.get('Preferred Stock')
        results['re'] = bs_2024.get('Retained Earnings')
        results['rect'] = bs_2024.get('Receivables')
        results['sale'] = is_2024.get('Total Revenue')
        results['lt'] = bs_2024.get('Total Liabilities Net Minority Interest')
        results['xint'] = is_2024.get('Interest Expense')
        results['ivao'] = bs_2024.get('Investments And Advances')

    except Exception as e:
        print(f"Error during variable extraction. Please check if the expected columns exist: {e}")
        return None

    # --- 3. Calculating the 7 Ratios ---
    try:
        # ch_rsst: Change in Revenue and Receivable
        results['ch_rsst'] = ((results['sale'] - results['rect']) /
                              (is_2023.get('Total Revenue') - bs_2023.get('Receivables')))

        # dch_rec: Change in Receivables
        results['dch_rec'] = (results['rect'] - bs_2023.get('Receivables')) / bs_2023.get('Total Assets')

        # ch_cs: Change in Common Stock
        results['ch_cs'] = (results['csho'] - bs_2023.get('Ordinary Shares Number')) / bs_2023.get('Ordinary Shares Number')

        # ch_cm: Change in Cash and Marketable Securities
        results['ch_cm'] = ((results['che'] + bs_2024.get('Cash Equivalents')) -
                            (bs_2023.get('Cash And Cash Equivalents') + bs_2023.get('Cash Equivalents'))) / results['at']

        # ch_roa: Change in Return on Assets
        roa_2024 = results['ni'] / results['at']
        roa_2023 = is_2023.get('Net Income Common Stockholders') / bs_2023.get('Total Assets')
        results['ch_roa'] = roa_2024 - roa_2023

        # dpi: Change in Gross Property, Plant, and Equipment
        results['dpi'] = (results['ppegt'] - bs_2023.get('Gross PPE')) / bs_2023.get('Gross PPE')

        # reoa: Retained Earnings to Assets
        results['reoa'] = results['re'] / results['at']

    except (KeyError, ZeroDivisionError) as e:
        print(f"Error during ratio calculation: {e}")
        return None

    return results

# Example usage
ticker_symbol = "GARAN.IS"
extracted_data = get_fraud_variables_and_ratios(ticker_symbol)
ticker_df = pd.DataFrame(extracted_data, index=[0])

if extracted_data:
    print(f"\n--- Extracted Variables and Calculated Ratios for {ticker_symbol} ---")
    for key, value in extracted_data.items():
        print(f"{key}: {value}")


--- Extracted Variables and Calculated Ratios for GARAN.IS ---
at: 3002579379000.0
ap: 110694236000.0
ceq: 329788449000.0
che: 591842159000.0
csho: 4200000000.0
dltt: 223968733000.0
dp: 3707320000.0
ni: 91243136000.0
ppegt: 47007468000.0
pstk: 0.0
re: 91279430000.0
rect: 57810685000.0
sale: 287168372000.0
lt: 2671170761000.0
xint: 414500152000.0
ivao: 314455248000.0
ch_rsst: 1.3685703597166776
dch_rec: 0.011843472275846187
ch_cs: 0.0
ch_cm: 0.039579443538168654
ch_roa: -0.008842562979160501
dpi: 0.5988956447559792
reoa: 0.030400338668282047


In [85]:
model_1 = XGBClassifier()
model_1.fit(X_train, y_train)

model_2 = DecisionTreeClassifier()
model_2.fit(X_train, y_train)

DecisionTreeClassifier()

In [86]:
for m in [model_1, model_2]:
    prediction = m.predict(ticker_df)
    prediction_proba = m.predict_proba(ticker_df)

    print(f"\n--- Prediction for SAHOL.IS ---")
    if prediction[0] == 1:
        print("The model predicts this is a FRAUDULENT financial statement.")
    else:
        print("The model predicts this is a NON-FRAUDULENT financial statement.")

    print(f"Fraud Probability: {prediction_proba[0][1]:.2f}")



--- Prediction for SAHOL.IS ---
The model predicts this is a NON-FRAUDULENT financial statement.
Fraud Probability: 0.00

--- Prediction for SAHOL.IS ---
The model predicts this is a FRAUDULENT financial statement.
Fraud Probability: 1.00
